In [2]:
import pathlib
import os

In [3]:
path = r"C:\Users\divya\OneDrive\Desktop\data allergy predict"
print(os.listdir(path))

['1. Eczema 1677', '10. Warts Molluscum and other Viral Infections - 2103', '2. Melanoma 15.75k', '3. Atopic Dermatitis - 1.25k', '4. Basal Cell Carcinoma (BCC) 3323', '5. Melanocytic Nevi (NV) - 7970', '6. Benign Keratosis-like Lesions (BKL) 2624', '7. Psoriasis pictures Lichen Planus and related diseases - 2k', '8. Seborrheic Keratoses and other Benign Tumors - 1.8k', '9. Tinea Ringworm Candidiasis and other Fungal Infections - 1.7k', 'app.py', 'requirements.txt', 'skin_cnn.ipynb', 'skin_cnn_model.keras', 'static', 'templates']


In [4]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Sequential
from tensorflow import keras

In [5]:
#dataset load ---- read images nd labels
data_dir = r"C:\Users\divya\OneDrive\Desktop\data allergy predict"

img_size = (180,180)         #Har image ko 180x180 size me convert karo
batch_size = 32             #Batch size = ek baar me 32 images model ko dene

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split = 0.2,
    subset = "training",
    seed = 42,
    image_size = img_size,
    batch_size = batch_size
)

#Tumhare folder ko scan karta hai
 #Har class ka name read karta hai
#images ko tensors (matrices) me convert karta hai
# Sabko resize karta hai 180 × 180
# 80% training ke liye deta hai
# 20% validation ke liye

Found 27155 files belonging to 12 classes.
Using 21724 files for training.


In [6]:
print("Class Names:", train_ds.class_names)

Class Names: ['1. Eczema 1677', '10. Warts Molluscum and other Viral Infections - 2103', '2. Melanoma 15.75k', '3. Atopic Dermatitis - 1.25k', '4. Basal Cell Carcinoma (BCC) 3323', '5. Melanocytic Nevi (NV) - 7970', '6. Benign Keratosis-like Lesions (BKL) 2624', '7. Psoriasis pictures Lichen Planus and related diseases - 2k', '8. Seborrheic Keratoses and other Benign Tumors - 1.8k', '9. Tinea Ringworm Candidiasis and other Fungal Infections - 1.7k', 'static', 'templates']


In [7]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split = 0.2,
    subset = "validation",
    seed = 42,
    image_size = img_size,
    batch_size = batch_size
)

##TensorFlow automatically does:

#Reads all images

#Detects folder names as labels

#Splits into train/validation

#Resizes images

#Creates batches

#Converts to tensors

Found 27155 files belonging to 12 classes.
Using 5431 files for validation.


In [ ]:
# --- INSERT THIS CODE SNIPPET ---
from sklearn.utils import class_weight
import numpy as np

print("Calculating balanced class weights...")

# 1. Extract labels from the training dataset
train_labels = np.concatenate([y for x, y in train_ds], axis=0).argmax(axis=1)

# 2. Calculate the weights inversely proportional to class frequencies
class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_labels),
    y=train_labels
)

# 3. Convert the array of weights into a dictionary expected by Keras
class_weights_dict = dict(enumerate(class_weights))

print("Computed Class Weights:", class_weights_dict)
# --- END INSERT ---
Use code with caution.

Step 3: Model Definition and Compilation (Already Done)
This section contains your model layers and compilation step (you likely have something similar to this):
python
# Model Definition (Add augmentation if you want)
model = keras.Sequential([
    # layers.RandomFlip("horizontal", input_shape=(180, 180, 3)), # Optional Augmentation
    # layers.RandomRotation(0.1),                                # Optional Augmentation
    layers.Rescaling(1./255, input_shape=(180,180,3)),
    # ... other Conv2D and MaxPooling layers ...
    layers.Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy', # Or 'categorical_crossentropy'
    metrics=['accuracy']
)
Use code with caution.

Step 4: Fit the Model (Modify Here)
Finally, use the class_weight argument in the model.fit() function:
python
# --- MODIFY THE model.fit() CALL ---

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=30, # Start with a higher number of epochs, e.g., 30
    class_weight=class_weights_dict # ADD THIS LINE
)

# --- SAVE THE NEW MODEL ---
model.save("skin_cnn_model.keras")
Use code with caution.






In [8]:
##CNN model

num_classes = len(train_ds.class_names)      #Dataset me kitni classes hai woh automatically count ho jayega.

model = keras.Sequential([                                                
    layers.Rescaling(1./255, input_shape=(180,180,3)),          

#Images pixels 0–255 hotay hain → Hume unhe 0–1 range me chahiye
#Isliye divide by 255 kiya.

layers.Conv2D(32,(3,3),activation="relu"),
layers.MaxPooling2D(),

#Conv2D: filter chala kar features nikalta hai (edges, textures)
#ReLU: negative values ko 0 banata hai
#MaxPooling: image size half kar deta hai → fast training
# Yaani yeh model ko features detect karna sikhata hai.

layers.Conv2D(64,(3,3),activation="relu"),
layers.MaxPool2D(),

#b 32 filter se increase karke 64 filter
#Feature zyada detailed milte hain
#(level 2 features: shapes, patterns)

layers.Conv2D(128,(3,3),activation="relu"),
layers.MaxPool2D(),

#e sabse deep features nikalta hai
#(level 3 features: disease texture, lesions pattern)

layers.Flatten(),
layers.Dense(128, activation="relu"),
layers.Dense(num_classes, activation="softmax")

])
#Flatten: 2D image ko 1D array bana deta hai
#Dense: decision making layer
#128 neurons: brain jaisa logic learn karta hai
#softmax: probabilities deta hai (kaunsi class hai)


model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=['accuracy']
)

model.summary()

#adam = sabse best optimizer
#loss = multi-class classification ke liye
#accuracy = performance show karega

c:\Users\divya\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\preprocessing\tf_data_layer.py:19: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ rescaling (Rescaling)           │ (None, 180, 180, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 178, 178, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 89, 89, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 87, 87, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 43, 43, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 41, 41, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 20, 20, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 51200)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     6,553,728 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 12)             │         1,548 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,648,524 (25.36 MB)

 Trainable params: 6,648,524 (25.36 MB)

 Non-trainable params: 0 (0.00 B)

In [9]:
#train the model
history = model.fit(
    train_ds,
    validation_data = val_ds,
    epochs = 10
)
#Model 10 baar dataset ko dekhega
#Har baar better hoga
#Har epoch pe accuracy change hogi
#Ye training process hota hai.

Epoch 1/10
679/679 ━━━━━━━━━━━━━━━━━━━━ 1179s 2s/step - accuracy: 0.4756 - loss: 1.4439 - val_accuracy: 0.5590 - val_loss: 1.1732
Epoch 2/10
679/679 ━━━━━━━━━━━━━━━━━━━━ 262s 386ms/step - accuracy: 0.5876 - loss: 1.0942 - val_accuracy: 0.5938 - val_loss: 1.0464
Epoch 3/10
679/679 ━━━━━━━━━━━━━━━━━━━━ 261s 384ms/step - accuracy: 0.6400 - loss: 0.9591 - val_accuracy: 0.6262 - val_loss: 0.9847
Epoch 4/10
679/679 ━━━━━━━━━━━━━━━━━━━━ 260s 383ms/step - accuracy: 0.6829 - loss: 0.8435 - val_accuracy: 0.6113 - val_loss: 1.0645
Epoch 5/10
679/679 ━━━━━━━━━━━━━━━━━━━━ 261s 384ms/step - accuracy: 0.7340 - loss: 0.7190 - val_accuracy: 0.6314 - val_loss: 1.0853
Epoch 6/10
679/679 ━━━━━━━━━━━━━━━━━━━━ 246s 361ms/step - accuracy: 0.7822 - loss: 0.6008 - val_accuracy: 0.6163 - val_loss: 1.3081
Epoch 7/10
679/679 ━━━━━━━━━━━━━━━━━━━━ 294s 433ms/step - accuracy: 0.8196 - loss: 0.4988 - val_accuracy: 0.6264 - val_loss: 1.2167
Epoch 8/10
679/679 ━━━━━━━━━━━━━━━━━━━━ 264s 389ms/step - accuracy: 0.8625 - l

In [10]:
model.save("skin_cnn_model.keras")       